In [8]:
import requests
import pandas as pd
import json
import datetime as dt

In [4]:
#requesting data for the price of rice in EU member states
riceprice=requests.get("https://www.ec.europa.eu/agrifood/api/rice/prices?")

In [34]:
# Print the result of the request as a python dictionary
euriceprice=riceprice.json()
print(df_riceprice.head())


  memberStateCode memberStateName   beginDate     endDate  weekNumber  \
0              IT           Italy  24/01/2005  30/01/2005          22   
1              IT           Italy  27/12/2004  02/01/2005          18   
2              IT           Italy  26/05/2008  01/06/2008          39   
3              IT           Italy  05/05/2008  11/05/2008          36   
4              IT           Italy  04/08/2008  10/08/2008          49   

      price   unit marketingYear   stage    type     variety  
0   €360.00  Tonne     2004/2005  Milled  Indica  Thaibonnet  
1   €360.00  Tonne     2004/2005  Milled  Indica  Thaibonnet  
2  €1000.00  Tonne     2007/2008  Milled  Indica  Thaibonnet  
3   €820.00  Tonne     2007/2008  Milled  Indica  Thaibonnet  
4  €1040.00  Tonne     2007/2008  Milled  Indica  Thaibonnet  


In [39]:
df_riceprice=pd.DataFrame(euriceprice)
df_riceprice["beginDate"]=pd.to_datetime(df_riceprice["beginDate"], format='%d%m%y');
#df_riceprice["endDate"]=pd.to_datetime(df_riceprice["endDate"], infer_datetime_format=True);


ValueError: time data '24/01/2005' does not match format '%d%m%y' (match)

In [32]:
#df_riceprice.set_index("beginDate", inplace=True)
#df_riceprice["2020-01-01":"2021-01-01"]